In [1]:
from json import dumps, loads
import time, os
import Queue
import threading 
import ast
import unittest
import requests
import random
import logging
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient

# Setup logging
logging.basicConfig(level=logging.INFO, filename="execution.log", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("Starting test_surge_bids")

# Setup variables
r = requests.get('http://10.129.6.52:4999/setup.in')
setup_str = r.text
setup_str = setup_str.split('\n')
NUM_SHARDS = int(setup_str[0])
NUM_REPLICAS = int(setup_str[1])
NUM_CLIENTS = int(setup_str[2])
logging.info("NUM_SHARDS: %s, NUM_REPLICAS: %s, NUM_CLIENTS: %s "%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS))
CS_HOST='10.129.6.52'
G = setup()[0]


In [2]:
global_client = ChainspaceClient(host=CS_HOST,port=5000)
init_transaction = surge.init()
init_tokens = init_transaction['transaction']['outputs']
global_client.process_transaction(init_transaction)
client_divs = surge.eq_div(NUM_CLIENTS, NUM_SHARDS)
print client_divs

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 3}", "{\"type\": \"InitToken\", \"location\": 3}", "{\"type\": \"InitToken\", \"location\": 3}", "{\"type\": \"InitToken\", \"location\": 4}", "{\"type\": \"InitToken\", \"location\": 4}", "{\"type\": \"InitToken\", \"location\": 4}", "{\"type\": \"InitToken\", \"location\": 5}", "{\"type\": \"InitToken

In [3]:
# Create prosumer clients for locations 
clients = []
base_port=5000


for s in range(0,NUM_SHARDS):
    clients.append([])
    for c in range(0,client_divs[s]):
        
#         clients[s].append(surge.SurgeClient(host=CS_HOST, port=base_port+s, init_token=init_tokens[idx]))
        clients[s].append(surge.SurgeClient(host=CS_HOST, port=base_port+s))

print clients

threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,client_divs[s]):
        client = clients[s][c]
        idx = sum(client_divs[:s]) + c
        threads[s].append(threading.Thread(target=client.create_surge_client, args=(init_tokens[idx],)) )

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

# c00 = surge.SurgeClient(5000, init_token[0]) # location=0
# Create sreps for locations 0 and 1 which will be clients in location 2
# rep0 = surge.SREPClient(5002, init_token[4]) # location=2

# Elect rep1 as SREP for location 0
# votes = (c00.cast_srep_vote(rep0.pub), c01.cast_srep_vote(rep0.pub))
# rep0.create_srep_client(5000, votes) 

# Elect rep2 as SREP for location 1
# votes = (c10.cast_srep_vote(rep1.pub), c11.cast_srep_vote(rep1.pub))
# rep1.create_srep_client(5001, votes) 


[[<chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836830>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836908>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f8368c0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836950>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836998>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f8369e0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836a28>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836a70>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836ab8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836b00>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836b48>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836b90>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f412f836bd8>, <chainspac

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_83e949bac3408c10c91444b1b49ff4933abbec244af185dc25b05379978b0398"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd0287d2c0943e5ba574b1f61465c30ca76c453271aef0e2c5543cfdcb1d", "92c71d002b3eea4dd719438cffde113daa59c7f148c21c1fc9ea1fa43b825ef327c71d002b90d621faebd7062b64e650e213b5f8a55086e54aadc75bc4c16d5b2b"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0287d2c0943e5ba574b1f61465c30ca76c453271aef0e2c5543cfdcb1d\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0287d2c0943e5ba574b1f61465c30ca76c453271aef0e2c5543cfdcb1d\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0287d2c0943e5ba574b1f61465c30ca76c453271aef0e2c5543cfdcb1d\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_83e949bac3408c10c91444b1b49ff4933abbec244af185dc25b

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
 POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_c724bd29ff09b4a6418f660f35c2104028c40401656be59646112c3d9307861f"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03e33243ba0a30829a223dc44a39014176b02dd00ec4969f606fdb46b4", "92c71d002be88dc8cdde807f25cd19844872d114cc8d378cef730353d6713daaa2c71d002b734c82332f9c8737be3d4816be2472687aeb5dc144ec01f2e9cb823e"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03e33243ba0a30829a223dc44a39014176b02dd00ec4969f606fdb46b4\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03e33243ba0a30829a223dc44a39014176b02dd00ec4969f606fdb46b4\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03e33243ba0a30829a223dc44a39014176b02dd00ec4969f606fdb46b4\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
 POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_3810fce58631d8fa893b8956471229d9c3f9922d116e07d746a70cd68c10e50e"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02352a106407c7d864e839373cc5792a6a2be0171283f22dbc39474f2b", "92c71d002b2d5e15183bf0cd330e4b67029e74b22aa7486ed8537f46ace635d9d5c71d002b8d80aa298622604ea7d1b4778652dceb65c87e5e182d10df240e2a5d"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02352a106407c7d864e839373cc5792a6a2be0171283f22dbc39474f2b\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02352a106407c7d864e839373cc5792a6a2be0171283f22dbc39474f2b\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02352a106407c7d864e839373cc5792a6a2be0171283f22dbc39474f2b\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_5eb08bf3c6b67d151011330189601bd915be11a6d93780964a1701891f3fa06c"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd025544749abf81340e1bfdf49ca7155c1c012786f13df786d0d7b13f7b", "92c71d002b7a0d1753fa21aca1dbc3faf010b79d1476780b38c3c8e6273292ba54c71d002be916c1bc09344a157a4263dd7154219b36a2bc4365332bdfaed6336a"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd025544749abf81340e1bfdf49ca7155c1c012786f13df786d0d7b13f7b\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd025544749abf81340e1bfdf49ca7155c1c012786f13df786d0d7b13f7b\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd025544749abf81340e1bfdf49ca7155c1c012786f13df786d0d7b13f7b\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_5eb08bf3c6b67d151011330189601bd915be11a6d93780964a1

...


{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


Error in sys.excepthook:
Traceback (most recent call last):


Original exception was:
Traceback (most recent call last):

Error in sys.excepthook:
Traceback (most recent call last):


Original exception was:
Traceback (most recent call last):



ZMQError: Too many open files

Error in sys.excepthook:
Traceback (most recent call last):


Original exception was:
Traceback (most recent call last):



HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OKHTTP/1.1 200 OK

{u'outcome': u'accepted_t_commit', u'success': u'True'}
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

In [4]:
# Create threads for bidding
threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,client_divs[s]):
        client = clients[s][c]
        bid_value = random.randint(0,100)
        threads[s].append(threading.Thread(target=client.submit_bid, args=(random.choice(['EBBuy', 'EBSell']),bid_value)) )

# trep0 = threading.Thread(target=rep0.accept_bids) 


In [5]:
# Run bidding threads
start = time.time()
for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

end = time.time()
duration = end-start
logging.info("Execution took "+ str(duration))
logging.info("TPS: %s"%(NUM_CLIENTS/duration))


POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_989867a196d1737266876005f87e32bebbbd87517b316cb4ea3c0bd9d70a5387"], "methodID": "submit_bid_proof", "parameters": ["EBBuy", "92c71d002b9c2acb1ebac4cbc689bf349d47ccf34b540c6263a3d02edb5f8d70e5c71d002b3768fdf14130997709c69b182a3e04c55c448805636f494abe1aed6c"], "outputs": ["{\"bid_type\": \"EBBuy\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02da8d38dae9ab325c61cd5368dba94971d04ee3710b9d32850f86af32\", \"quantity_sig\": \"92c71d002bea1f4182d810fb750136d0579430b1c162c77ba2a9d44d2437c38de5c71d002b13adb2fa4e812fc4c450fb5c54b7773c88a3ebcd9db029825060e0f0\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02da8d38dae9ab325c61cd5368dba94971d04ee3710b9d32850f86af32\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_989867a196d1737266876005f87e32bebbbd87517b316cb4ea3c0bd9d70a5387": "{\"type\": \"EBTok

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_d5bba4672c834156d94acc7e7f2204c0131b3a29072a43b1def793b667e60607"], "methodID": "submit_bid_proof", "parameters": ["EBSell", "92c71d002b0e2dbeab45a39bc43ce3aed905abc4ec8cdbf47bbd419ee68758806dc71d002b54727f2b5a33f608ef8ed0df518453aac5a914712489175f13789675"], "outputs": ["{\"bid_type\": \"EBSell\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0343ddf455cad4a4ccd22e040ad07a937d2265264e70bc0e0ec3387cf4\", \"quantity_sig\": \"92c71d002b55972ffa18e3bb8130f9fd74e83586d38bab6cc023ba8e167cce65cec71d002b754b7f72b8eb31022c43c05da8d9d54f88be5cefcc0a880306fb2efa\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0343ddf455cad4a4ccd22e040ad07a937d2265264e70bc0e0ec3387cf4\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_d5bba4672c834156d94acc7e7f2204c0131b3a29072a43b1def793b667e60607": "{\"type\": \"EBT

{"transaction": {"inputIDs": ["0_e416bdce365bb9311cb4c716382d5f2d88a50f0597715e9ce6b94303edaa17ac"], "methodID": "submit_bid_proof", "parameters": ["EBSell", "92c71d002b2e1790bcf87c5c5eb154cfc61844bf12b7e3971c3794f5b20878629fc71d002b2c9d8e7ac549235b47911ebd69aa5d672815f1dd457f7aa331d5dcbc"], "outputs": ["{\"bid_type\": \"EBSell\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03bc8f6711beaf604f5889278ea8ed2369f23ae30e8ae67b74699a1fc4\", \"quantity_sig\": \"92c71d002b6d550065a52299eb5fe119edad74334b709c495b3f171c111e507a9ec71d002ba13acd1d7cf5951b6c30ad31adc7f8b76942bc4e36a3db25c3dfd790\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03bc8f6711beaf604f5889278ea8ed2369f23ae30e8ae67b74699a1fc4\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_e416bdce365bb9311cb4c716382d5f2d88a50f0597715e9ce6b94303edaa17ac": "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03bc8f6711beaf

 POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_f03d2978156b609c380ff9ce5536e19fedce97fa3ee9e61c703c996289234324"], "methodID": "submit_bid_proof", "parameters": ["EBSell", "92c71d002ba6de6b0b07de442ce15da0b934a4941def30d32af2829f806aa07375c71d002b0ddb05fd85d2ea87421d6ce1ca61586e71100bcf3268a0d4ab011b0f"], "outputs": ["{\"bid_type\": \"EBSell\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02256d3f864ec7c1d7fe4c8addc6407a5a76907c424a175d9712220b7e\", \"quantity_sig\": \"92c71d002b38cd746b4d2df3ff97a802c33615b044f75d2c5de6679e69c0f43054c71d002be36675debe2b60ba758cbd0db9e499a6e70dc6c1a0c734a2da65f43a\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02256d3f864ec7c1d7fe4c8addc6407a5a76907c424a175d9712220b7e\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_f03d2978156b609c380ff9ce5536e19fedce97fa3ee9e61c703c996289234324": "{\"type\": \"EB

{"transaction": {"inputIDs": ["0_ced7f0bccd549a3f463df6d0a683b8091b250d7c5b7f5a95291aa4d3dd140d17"], "methodID": "submit_bid_proof", "parameters": ["EBBuy", "92c71d002b50e3f0693abe7b32fec24be3ae1d233273a7b8a4a38e17bfeca8f4b4c71d002b7f9dd84a15f6873edd38a8871cca3fffce5a31fcc6bbd539f60cf268"], "outputs": ["{\"bid_type\": \"EBBuy\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03d2a3dbd065333d8ed6f1497ba02a6ec6a3ec4a386f5823454704a2c9\", \"quantity_sig\": \"92c71d002b83c8acd05b58598ea6c284ced0fe786bc218f69b5f2134b481faadc6c71d002bd428626e2be5ced70dadc1c868e14a6045fb678682f98761d15d8113\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03d2a3dbd065333d8ed6f1497ba02a6ec6a3ec4a386f5823454704a2c9\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_ced7f0bccd549a3f463df6d0a683b8091b250d7c5b7f5a95291aa4d3dd140d17": "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03d2a3dbd065333d

...

ZMQError: Too many open files


{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


ZMQError: Too many open files

ZMQError: Too many open files

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

In [6]:
# '{}|{}'.format(50, loads(r1.ebtoken)['pub'])
# global_client.fix_json("{'type':'EBBuy','location':0")
# global_client = ChainspaceClient(port=5001)
# global_client.get_objects({'location':0,'type':'BidAccept'})
# print buy_bids

# print loads(objs[0])['type']
# bid_proofs = rep1.client.get_objects({ 'type':'EBBuy'})
# print init_token1
